In [283]:
import re
import numpy as np
import pandas as pd
from collections import defaultdict
from itertools import permutations

In [284]:
f = open("voters.txt")
txt = f.read()
# txt = re.sub(r'[{}]', '', txt)

In [285]:
preferences = [val.split(': ')[-1] for val in txt.split('\n')]
counts = [float(val.split(': ')[0]) for val in txt.split('\n')]

In [286]:
preferences_to_counts = {preference: count for preference, count in zip(preferences, counts)}
preferences_to_counts = defaultdict(int, preferences_to_counts)


new_preferece_counts = []
preferences_with_ties = []
for preference, count in zip(preferences, counts):
    ties = re.findall("{\d+,\d+}", preference)
    n_ties = len(ties)
    if(n_ties>0):
        print(preference)
        preferences_with_ties.append(preference)
#       we expect only one tie for each preference of alternatives
        tie = re.sub("[{}]", "", ties[0]).split(',')
        for t in tie:
            new_preferece_counts.append((re.sub(re.escape(ties[0]), t, preference), count/2))
            
for preference, extra_counts in new_preferece_counts:
    preferences_to_counts[preference] += extra_counts
for preference in preferences_with_ties:
    del preferences_to_counts[preference]

3,4,5,6,7,8,9,{1,10},2
2,4,5,7,{9,10},3,6,1,8
1,2,6,5,3,7,8,9,{10,11}
4,2,5,3,1,6,7,8,{9,10}
9,3,2,5,{1,4},6,7,8
2,8,3,4,{1,5},6,9,7
3,5,1,{7,9},8,6,4,2
3,1,2,5,6,8,{9,10}
2,4,1,{5,8},6,9


In [287]:
preferences_to_counts

defaultdict(int,
            {'3,8': 31.0,
             '2,4': 30.0,
             '3': 27.0,
             '4,2': 24.0,
             '8': 23.0,
             '4,8': 21.0,
             '4': 21.0,
             '8,4': 17.0,
             '2,5': 16.0,
             '1,8': 14.0,
             '2': 14.0,
             '8,1': 13.0,
             '5,8': 13.0,
             '8,3': 12.0,
             '5,1': 12.0,
             '5,7': 12.0,
             '4,3': 10.0,
             '4,5': 10.0,
             '2,5,4': 9.0,
             '5,2': 9.0,
             '3,5': 9.0,
             '1,5': 9.0,
             '2,7': 9.0,
             '3,1': 9.0,
             '8,5': 9.0,
             '7': 9.0,
             '4,7': 8.0,
             '3,8,4': 7.0,
             '2,1': 7.0,
             '8,2': 7.0,
             '3,4': 7.0,
             '5,4': 7.0,
             '4,2,1': 6.0,
             '3,1,8': 6.0,
             '3,7': 6.0,
             '2,8': 6.0,
             '4,1': 6.0,
             '2,4,5,1': 5.0,
             

In [288]:
alternatives = [str(i) for i in range(1, 12)]
alternatives

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']

In [289]:
preferences = [preference.split(',') for preference in list(preferences_to_counts.keys())]
counts = list(preferences_to_counts.values())

In [290]:
df =  pd.DataFrame([counts, preferences]).T
df.rename(columns={0: 'n', 1: 'preference'}, inplace=True)

In [291]:
# has +9 new preferences due to the ties introduced
df.tail(18)

,n,preference
1751,0.5,"[3, 4, 5, 6, 7, 8, 9, 1, 2]"
1752,0.5,"[3, 4, 5, 6, 7, 8, 9, 10, 2]"
1753,0.5,"[2, 4, 5, 7, 9, 3, 6, 1, 8]"
1754,0.5,"[2, 4, 5, 7, 10, 3, 6, 1, 8]"
1755,0.5,"[1, 2, 6, 5, 3, 7, 8, 9, 10]"
1756,0.5,"[1, 2, 6, 5, 3, 7, 8, 9, 11]"
1757,0.5,"[4, 2, 5, 3, 1, 6, 7, 8, 9]"
1758,0.5,"[4, 2, 5, 3, 1, 6, 7, 8, 10]"
1759,0.5,"[9, 3, 2, 5, 1, 6, 7, 8]"
1760,0.5,"[9, 3, 2, 5, 4, 6, 7, 8]"


In [292]:
# number of total voters
df.n.sum()

2477.0

In [293]:
# 
df = df.sample(frac =.10, random_state=42)

In [294]:
def get_p_scores(df, alternatives):
    scores = {}
    for a in alternatives:
        scores[a] = 0
    for index, row in df.iterrows():
        if(len(row.preference) > 0):
            scores[row.preference[0]] += int(row.n)
    return scores

def get_score_alternatives(scores, mode):
    mode_score = min(scores, key=scores.get)
    if(mode == 'max'):
        mode_score = max(scores, key=scores.get)
    mode_score_alternatives = [i for i, val in scores.items() if val == scores[mode_score]]
    return mode_score_alternatives

def remove_alternatives(df, alternatives):
    df.preference = df.apply(lambda x: list(set(x.preference).difference(set(alternatives))), axis=1)
    return df

def STV(df, alternatives, verbose=1):
    votes_df = df.copy()
    scores = get_p_scores(votes_df, alternatives)
    while(len(alternatives)>0):
        scores = get_p_scores(votes_df, alternatives)
        min_p_alternatives = get_score_alternatives(scores, 'min')
        alternatives = list(set(alternatives).difference(set(min_p_alternatives)))
        if(len(alternatives)==0):
            if(verbose > 0):
                print("Reached a decision")
            break
        if(verbose > 0):
            print(scores)
            print('removing alternatives', min_p_alternatives)
        votes_df = remove_alternatives(votes_df, min_p_alternatives)
        if(verbose > 1):
            print(votes_df)
    return list(scores.keys())

In [295]:
winners = STV(df, alternatives)
winners

{'1': 26, '2': 33, '3': 56, '4': 21, '5': 27, '6': 6, '7': 9, '8': 42, '9': 2, '10': 0, '11': 0}
removing alternatives ['10', '11']
{'1': 67, '6': 47, '7': 10, '2': 3, '5': 40, '8': 46, '9': 2, '4': 7, '3': 0}
removing alternatives ['3']
{'6': 35, '8': 36, '7': 12, '2': 1, '5': 48, '1': 81, '9': 2, '4': 7}
removing alternatives ['2']
{'1': 56, '6': 56, '7': 10, '5': 57, '8': 34, '9': 2, '4': 7}
removing alternatives ['9']
{'1': 55, '6': 54, '7': 9, '5': 61, '8': 34, '4': 8}
removing alternatives ['4']
{'1': 48, '6': 51, '7': 9, '5': 75, '8': 30}
removing alternatives ['7']
{'5': 153, '8': 31, '6': 0, '1': 24}
removing alternatives ['6']
{'5': 153, '8': 31, '1': 24}
removing alternatives ['1']
{'5': 153, '8': 37}
removing alternatives ['8']
Reached a decision


['5']

In [315]:
def consider_alternative(x):
    w_found = 0
    winner_in = [False for winner in winners]
    winner_idx = [-1 for winner in winners]
    for idx, winner in enumerate(winners):
        if(winner in x.preference):
            winner_in[idx] = True
            winner_idx[idx] = x.preference.index(winner)
            w_found += 1
    for in_, idx in zip(winner_in, winner_idx):
        if(in_ and idx < 2 and idx >=0):
            return False
    if(w_found > 0 and (len(x.preference) - w_found) > 1): 
        return min(winner_idx) 
    if(len(x.preference) > 1):
        return len(x.preference)
    else:
        return False

In [316]:
df[f'consider'] = df.apply(consider_alternative, axis=1)
df_filtered = df[df['consider'] != False]
df_filtered = df_filtered.sort_values(by=['n', 'consider'])

In [317]:
df_filtered.head(10)

,n,preference,consider
1767,0.5,"[2, 4, 1, 5, 6, 9]",3
1761,0.5,"[2, 8, 3, 4, 1, 6, 9, 7]",8
1544,1.0,"[7, 8, 5, 6]",2
237,1.0,"[3, 8, 5, 2, 1, 4, 7, 6, 9]",2
425,1.0,"[8, 3, 5, 1, 7, 9, 6, 4, 2]",2
567,1.0,"[3, 7, 5, 9, 8, 6, 4, 2, 1]",2
297,1.0,"[2, 4, 5, 7, 8, 9, 6, 3, 1]",2
741,1.0,"[2, 4, 5, 3, 6, 8, 1, 9, 7]",2
1216,1.0,"[2, 4, 5, 1, 7]",2
1531,1.0,"[8, 2, 5, 4]",2


In [318]:
df_filtered

,n,preference,consider
1767,0.5,"[2, 4, 1, 5, 6, 9]",3
1761,0.5,"[2, 8, 3, 4, 1, 6, 9, 7]",8
1544,1.0,"[7, 8, 5, 6]",2
237,1.0,"[3, 8, 5, 2, 1, 4, 7, 6, 9]",2
425,1.0,"[8, 3, 5, 1, 7, 9, 6, 4, 2]",2
...,...,...,...
69,3.0,"[3, 8, 2, 4]",4
49,4.0,"[3, 8, 7]",3
29,7.0,"[8, 2]",2
30,7.0,"[3, 4]",2


In [319]:
df_filtered.consider.value_counts()

3    45
2    28
4    19
5    18
6     7
7     7
8     5
Name: consider, dtype: int64

In [320]:
def create_alternatives(x):
    # return x.preference[:x.consider]
    orderings = [list(item) + x.preference[x.consider:] for item in list(permutations(x.preference[:x.consider]))[1:]]
    # orderings.remove(x.preference[:x.consider])
    return orderings

In [321]:
df_filtered.loc[:,'list'] = df_filtered.apply(create_alternatives, axis=1)
df_filtered.loc[:,'list_counts'] = df_filtered.apply(lambda x: len(x.list), axis=1)
df_filtered

,n,preference,consider,list,list_counts
1767,0.5,"[2, 4, 1, 5, 6, 9]",3,"[[2, 1, 4, 5, 6, 9], [4, 2, 1, 5, 6, 9], [4, 1...",5
1761,0.5,"[2, 8, 3, 4, 1, 6, 9, 7]",8,"[[2, 8, 3, 4, 1, 6, 7, 9], [2, 8, 3, 4, 1, 9, ...",40319
1544,1.0,"[7, 8, 5, 6]",2,"[[8, 7, 5, 6]]",1
237,1.0,"[3, 8, 5, 2, 1, 4, 7, 6, 9]",2,"[[8, 3, 5, 2, 1, 4, 7, 6, 9]]",1
425,1.0,"[8, 3, 5, 1, 7, 9, 6, 4, 2]",2,"[[3, 8, 5, 1, 7, 9, 6, 4, 2]]",1
...,...,...,...,...,...
69,3.0,"[3, 8, 2, 4]",4,"[[3, 8, 4, 2], [3, 2, 8, 4], [3, 2, 4, 8], [3,...",23
49,4.0,"[3, 8, 7]",3,"[[3, 7, 8], [8, 3, 7], [8, 7, 3], [7, 3, 8], [...",5
29,7.0,"[8, 2]",2,"[[2, 8]]",1
30,7.0,"[3, 4]",2,"[[4, 3]]",1


In [322]:
df_filtered = df_filtered[df_filtered['consider'] == 8]
df_filtered

,n,preference,consider,list,list_counts
1761,0.5,"[2, 8, 3, 4, 1, 6, 9, 7]",8,"[[2, 8, 3, 4, 1, 6, 7, 9], [2, 8, 3, 4, 1, 9, ...",40319
552,1.0,"[6, 8, 9, 7, 3, 1, 4, 2, 5]",8,"[[6, 8, 9, 7, 3, 1, 2, 4, 5], [6, 8, 9, 7, 3, ...",40319
943,1.0,"[3, 8, 6, 9, 1, 2, 4, 7]",8,"[[3, 8, 6, 9, 1, 2, 7, 4], [3, 8, 6, 9, 1, 4, ...",40319
873,1.0,"[4, 8, 1, 2, 3, 7, 6, 9, 5]",8,"[[4, 8, 1, 2, 3, 7, 9, 6, 5], [4, 8, 1, 2, 3, ...",40319
990,1.0,"[3, 8, 7, 6, 2, 1, 9, 4]",8,"[[3, 8, 7, 6, 2, 1, 4, 9], [3, 8, 7, 6, 2, 9, ...",40319


In [323]:
df_filtered.loc[:,'preference_s'] = df_filtered.loc[:,'preference'].astype(str)

C:\Users\jl\AppData\Local\Temp\ipykernel_5556\3900475207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:,'preference_s'] = df_filtered.loc[:,'preference'].astype(str)


In [324]:
df_filtered = df_filtered.set_index('preference_s')
df_filtered

,n,preference,consider,list,list_counts
preference_s,,,,,
"['2', '8', '3', '4', '1', '6', '9', '7']",0.5,"[2, 8, 3, 4, 1, 6, 9, 7]",8,"[[2, 8, 3, 4, 1, 6, 7, 9], [2, 8, 3, 4, 1, 9, ...",40319
"['6', '8', '9', '7', '3', '1', '4', '2', '5']",1.0,"[6, 8, 9, 7, 3, 1, 4, 2, 5]",8,"[[6, 8, 9, 7, 3, 1, 2, 4, 5], [6, 8, 9, 7, 3, ...",40319
"['3', '8', '6', '9', '1', '2', '4', '7']",1.0,"[3, 8, 6, 9, 1, 2, 4, 7]",8,"[[3, 8, 6, 9, 1, 2, 7, 4], [3, 8, 6, 9, 1, 4, ...",40319
"['4', '8', '1', '2', '3', '7', '6', '9', '5']",1.0,"[4, 8, 1, 2, 3, 7, 6, 9, 5]",8,"[[4, 8, 1, 2, 3, 7, 9, 6, 5], [4, 8, 1, 2, 3, ...",40319
"['3', '8', '7', '6', '2', '1', '9', '4']",1.0,"[3, 8, 7, 6, 2, 1, 9, 4]",8,"[[3, 8, 7, 6, 2, 1, 4, 9], [3, 8, 7, 6, 2, 9, ...",40319


In [325]:
df_filtered.iloc[0].list

[['2', '8', '3', '4', '1', '6', '7', '9'],
 ['2', '8', '3', '4', '1', '9', '6', '7'],
 ['2', '8', '3', '4', '1', '9', '7', '6'],
 ['2', '8', '3', '4', '1', '7', '6', '9'],
 ['2', '8', '3', '4', '1', '7', '9', '6'],
 ['2', '8', '3', '4', '6', '1', '9', '7'],
 ['2', '8', '3', '4', '6', '1', '7', '9'],
 ['2', '8', '3', '4', '6', '9', '1', '7'],
 ['2', '8', '3', '4', '6', '9', '7', '1'],
 ['2', '8', '3', '4', '6', '7', '1', '9'],
 ['2', '8', '3', '4', '6', '7', '9', '1'],
 ['2', '8', '3', '4', '9', '1', '6', '7'],
 ['2', '8', '3', '4', '9', '1', '7', '6'],
 ['2', '8', '3', '4', '9', '6', '1', '7'],
 ['2', '8', '3', '4', '9', '6', '7', '1'],
 ['2', '8', '3', '4', '9', '7', '1', '6'],
 ['2', '8', '3', '4', '9', '7', '6', '1'],
 ['2', '8', '3', '4', '7', '1', '6', '9'],
 ['2', '8', '3', '4', '7', '1', '9', '6'],
 ['2', '8', '3', '4', '7', '6', '1', '9'],
 ['2', '8', '3', '4', '7', '6', '9', '1'],
 ['2', '8', '3', '4', '7', '9', '1', '6'],
 ['2', '8', '3', '4', '7', '9', '6', '1'],
 ['2', '8',

In [326]:
re.sub('[\]\[\']', '', df_filtered.index.to_list()[0]).split(', ')

['2', '8', '3', '4', '1', '6', '9', '7']

In [327]:
df_filtered.loc[df_filtered.index.to_list()[0],:]

n                                                            0.5
preference                              [2, 8, 3, 4, 1, 6, 9, 7]
consider                                                       8
list           [[2, 8, 3, 4, 1, 6, 7, 9], [2, 8, 3, 4, 1, 9, ...
list_counts                                                40319
Name: ['2', '8', '3', '4', '1', '6', '9', '7'], dtype: object

In [328]:
'asa' in df_filtered.index

False

In [329]:
def get_smallest_change():
    for index, row in df_filtered.iterrows():
        print(index)
        for new_combination in row.list:
            new_df = df_filtered.copy()
            try:
                if str(new_combination) in new_df.index:
                    new_df.loc[str(new_combination), 'n'] += row.n
                else:
                    new_df.loc[str(new_combination)] = [row.n, new_combination, -1, [], -1]
                new_df.drop(str(new_combination))
                new_winners = STV(new_df, alternatives, verbose=0)
                if(set(winners) == set(new_winners)):
                    return row, new_combination
            except Exception as e:
                print(e)
                return new_df, new_combination
    return 'not found', 'not_found'
info, preferences = get_smallest_change()

['2', '8', '3', '4', '1', '6', '9', '7']
['6', '8', '9', '7', '3', '1', '4', '2', '5']
['3', '8', '6', '9', '1', '2', '4', '7']
['4', '8', '1', '2', '3', '7', '6', '9', '5']
['3', '8', '7', '6', '2', '1', '9', '4']


In [330]:
info

'not found'

In [331]:
preferences

'not_found'